# ML-3 Обучение с учителем. Классификация
###  Содержание <a class="anchor" id=0></a>
- [1. Введение](#1)
- [2. Логическая регрессия](#2)
- [3. Метрики классификации. Мультиклассовая классификация](#3)
- [4. Логическия регрессия. Практика](#4)
- [5. Деревья решений](#5)
- [6. Введение в ансамбли: бэггинг. Случайные лес](#6)
- [7. Деревья решений и случайный лес. Практика](#7)
- [8. Итоги](#8)

# 1. Введение <a class="anchor" id=1></a>

[к содержанию](#0)

# 2. Логическая регрессия <a class="anchor" id=2></a>

[к содержанию](#0)

Что вообще означает `«решить задачу классификации»`? Это значит **построить разделяющую поверхность в пространстве признаков**, которая делит пространство на части, каждая из которых соответствует определённому классу. 

Ниже представлены примеры разделяющих поверхностей, которые производят бинарную классификацию. Красным и синим цветом обозначены классы, зелёным — собственно поверхность, которая делит пространство признаков на две части. В каждой из этих частей находятся только наблюдения определённого класса.

<img src=ml3_img1.png>

Модели, которые решают задачу классификации, называются классификаторами (`classifier`).

Если взять в качестве разделяющей поверхности некоторую плоскость (ровная поверхность на первом рисунке), то мы получаем модель логистической регрессии, которая тесно связана с рассмотренной нами ранее линейной регрессией.

Давайте для начала вспомним, как выглядит уравнение модели линейной регрессии в общем случае:

<img src=ml3_img2.png>

>Но всё это работает только в том случае, когда целевой признак `y`, который мы хотим предсказать, является числовым, например цена, вес, время аренды и т. д.

Что же делать с этой моделью, когда целевой признак `y` является категориальным? Например, является письмо спамом или обычным письмом?

Можно предположить, что, раз у нас есть две категории, мы можем обозначить категории за `y = 1` (Спам) и `y = 0` (Не спам) и обучить линейную регрессию предсказывать `0` и `1`.

Но результат будет очень плохим. Выглядеть это будет примерно так:

<img src=ml3_img3.png>

Для больших значений `x` прямая будет выдавать значения **больше 1**, а для очень маленьких — **меньше 0**. Что это значит? Непонятно. Непонятно и то, что делать со значениями в диапазоне от 0 до 1. Да, можно относить значения на прямой выше 0.5 к классу 1, а меньше либо равным 0.5 — к классу 0, но это всё «костыли».

>Идея! Давайте переведём задачу классификации в задачу регрессии. Вместо предсказания класса будем предсказывать вероятность принадлежности к этому классу. 

Модель должна выдавать некоторую вероятность `P`, которая будет определять, принадлежит ли данный объект к классу 1: например, вероятность того, что письмо является спамом. При этом вероятность того, что письмо является обычным письмом (класс 0), определяется как `Q = 1 - P`.  

Когда модель будет обучена на предсказание вероятности, мы зададим некоторый порог вероятности. Если предсказанная вероятность будет выше этого порога, мы определим объект к классу 1, а если ниже — к классу 0.

Например, стандартный порог равен `0.5`. То есть если вероятность `P > 0.5`, мы будем считать письмо спамом, а если `P <= 0.5` — обычным информативным письмом.

В итоге мы добьёмся того, что будем предсказывать не дискретный категориальный, а непрерывный числовой признак, который лежит в диапазоне `[0, 1]`. А это уже знакомая нам задача регрессии.

>Однако остался главный вопрос: как научить модель предсказывать вероятности, ведь они должны лежать строго в диапазоне от 0 до 1, а предсказания линейной регрессии лежат в диапазоне от `-inf` до `+inf`? 

Тут-то мы и приходим к модели логистической регрессии — **регрессии вероятностей**.

# 3. Метрики классификации. Мультиклассовая классификация <a class="anchor" id=3></a>

[к содержанию](#0)

# 4. Логическия регрессия. Практика <a class="anchor" id=4></a>

[к содержанию](#0)

# 5. Деревья решений <a class="anchor" id=5></a>

[к содержанию](#0)

# 6. Введение в ансамбли: бэггинг. Случайные лес <a class="anchor" id=6></a>

[к содержанию](#0)

# 7. Деревья решений и случайный лес. Практика <a class="anchor" id=7></a>

[к содержанию](#0)

# 8. Итоги <a class="anchor" id=8></a>

[к содержанию](#0)